# Imports

In [132]:
import pandas as pd
import numpy as np

# Reading The Datasets

In [133]:
train=pd.read_parquet('../Data/feature_removed_small_train.parquet',engine='pyarrow')
val=pd.read_parquet('../Data/feature_removed_small_val.parquet',engine='pyarrow')

In [134]:
train

,id2,id3,id4,id5,y,f1,f2,f5,f6,f7,...,f366,f349_sin,time_since_last,session_event_count,offer_category,num_offer_categories,sub_category,num_sub_categories,previous_offer_category,previous_suboffer_category
0,1000624,5243695,2023-11-01 03:31:21.598,2023-11-01,0,56.0,NaN,33.0,16.0,33.0,...,0.000000,-0.433884,NaN,1,Shopping,1,Other,1,missing,missing
1,1000624,97646,2023-11-01 03:31:21.618,2023-11-01,0,56.0,NaN,33.0,16.0,33.0,...,0.000000,-0.433884,0.020,2,Shopping,1,Apparel,1,Shopping,Other
2,1000639,81010,2023-11-01 13:00:17.848,2023-11-01,0,44.0,NaN,24.0,46.0,42.0,...,0.005056,-0.433884,NaN,1,Shopping,1,Other,1,missing,missing
3,1000639,830157,2023-11-01 13:00:20.013,2023-11-01,0,44.0,NaN,24.0,46.0,42.0,...,0.005056,-0.433884,2.165,2,Shopping,1,Gifts,1,Shopping,Other
4,1000639,24840,2023-11-01 13:00:22.564,2023-11-01,0,44.0,NaN,24.0,46.0,42.0,...,0.007229,-0.433884,2.551,3,Shopping,1,Clothing_Gifts_Jewelry,3,Shopping,Gifts
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124373,1910541,80107221,2023-11-01 05:46:56.017,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,0.000000,-0.433884,1.993,3,Shopping,1,Other,1,Services,None
124374,1910541,224505,2023-11-01 07:43:02.618,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,0.000000,-0.433884,6966.601,4,Shopping,1,Other,1,Shopping,Other
124375,1910541,5420674,2023-11-01 07:43:31.173,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,0.000000,-0.433884,28.555,5,Shopping,1,Other,1,Shopping,Other
124376,1910541,82192,2023-11-01 07:43:31.452,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,0.000000,-0.433884,0.279,6,Shopping,1,Clothing,1,Shopping,Other


# Sorting by customer ID and time

In [135]:

# CRITICAL: Ensure your DataFrame is sorted by user and time first
train = train.sort_values(['id2', 'id4'])
val = val.sort_values(['id2', 'id4'])


# Feature 1: No of clicks previously

In [136]:

# --- 1. Calculate 'no_of_clicks_previously' ---
# This is the cumulative sum of clicks for each user, shifted by 1
# to ensure it only includes PAST clicks and avoids data leakage.
train['no_of_clicks_previously'] = train.groupby('id2')['y'].cumsum().shift(1)
val['no_of_clicks_previously'] = val.groupby('id2')['y'].cumsum().shift(1)


/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_77493/1493248926.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train['no_of_clicks_previously'] = train.groupby('id2')['y'].cumsum().shift(1)
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_77493/1493248926.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val['no_of_clicks_previously'] = val.groupby('id2')['y'].cumsum().shift(1)


In [137]:
train[train['y']==1]

,id2,id3,id4,id5,y,f1,f2,f5,f6,f7,...,f349_sin,time_since_last,session_event_count,offer_category,num_offer_categories,sub_category,num_sub_categories,previous_offer_category,previous_suboffer_category,no_of_clicks_previously
253,1002750,64187,2023-11-01 05:21:58.000709,2023-11-01,1,NaN,4.0,29.0,29.0,34.0,...,-0.433884,2.092709,49,Dining_Entertainment,2,Other,1,Shopping,Other,0.0
326,1002750,857184,2023-11-01 05:28:01.000106,2023-11-01,1,NaN,4.0,29.0,29.0,34.0,...,-0.433884,7.109106,122,Dining,1,Other,1,Shopping,Clothing,1.0
457,1003324,399752,2023-11-02 07:11:32.000766,2023-11-02,1,76.0,40.0,92.0,48.0,37.0,...,-0.974928,1.591766,44,Shopping,1,Apparel,1,Shopping,Other,0.0
527,1004168,498907,2023-11-01 14:01:41.000699,2023-11-01,1,NaN,NaN,38.0,NaN,NaN,...,-0.433884,12.449699,21,Travel,1,Hotel resorts,1,Shopping,Apparel_Clothing_Gifts_Jewelry_Other shopping,0.0
605,1004637,98107,2023-11-02 06:45:15.000821,2023-11-02,1,NaN,38.0,6.0,14.0,NaN,...,-0.974928,5.459821,21,Dining,1,Other,1,Shopping,Other,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124240,1910264,876665835,2023-11-01 16:08:14.000362,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,-0.433884,13.312362,14,Shopping,1,Other,1,Shopping,Clothing,0.0
124241,1910264,401464,2023-11-01 16:08:41.000508,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,-0.433884,27.000146,15,Shopping,1,Auto fuel,1,Shopping,Other,1.0
124317,1910264,81010,2023-11-01 16:19:50.000217,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,-0.433884,7.848217,91,Shopping,1,Other,1,Shopping,Other,2.0
124336,1910264,958700,2023-11-01 16:23:59.000189,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,-0.433884,10.611189,110,Entertainment,1,Other,1,Shopping,Jewelry,3.0


In [138]:

# For the first event of each user, the shift creates a NaN. We'll fill this with 0.
# We also need to make sure the shift doesn't carry over from the previous user.
train.loc[train['id2'] != train['id2'].shift(1), 'no_of_clicks_previously'] = 0
val.loc[val['id2'] != val['id2'].shift(1), 'no_of_clicks_previously'] = 0

In [139]:
train

,id2,id3,id4,id5,y,f1,f2,f5,f6,f7,...,f349_sin,time_since_last,session_event_count,offer_category,num_offer_categories,sub_category,num_sub_categories,previous_offer_category,previous_suboffer_category,no_of_clicks_previously
0,1000624,5243695,2023-11-01 03:31:21.598,2023-11-01,0,56.0,NaN,33.0,16.0,33.0,...,-0.433884,NaN,1,Shopping,1,Other,1,missing,missing,0.0
1,1000624,97646,2023-11-01 03:31:21.618,2023-11-01,0,56.0,NaN,33.0,16.0,33.0,...,-0.433884,0.020,2,Shopping,1,Apparel,1,Shopping,Other,0.0
2,1000639,81010,2023-11-01 13:00:17.848,2023-11-01,0,44.0,NaN,24.0,46.0,42.0,...,-0.433884,NaN,1,Shopping,1,Other,1,missing,missing,0.0
3,1000639,830157,2023-11-01 13:00:20.013,2023-11-01,0,44.0,NaN,24.0,46.0,42.0,...,-0.433884,2.165,2,Shopping,1,Gifts,1,Shopping,Other,0.0
4,1000639,24840,2023-11-01 13:00:22.564,2023-11-01,0,44.0,NaN,24.0,46.0,42.0,...,-0.433884,2.551,3,Shopping,1,Clothing_Gifts_Jewelry,3,Shopping,Gifts,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124373,1910541,80107221,2023-11-01 05:46:56.017,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,-0.433884,1.993,3,Shopping,1,Other,1,Services,None,0.0
124374,1910541,224505,2023-11-01 07:43:02.618,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,-0.433884,6966.601,4,Shopping,1,Other,1,Shopping,Other,0.0
124375,1910541,5420674,2023-11-01 07:43:31.173,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,-0.433884,28.555,5,Shopping,1,Other,1,Shopping,Other,0.0
124376,1910541,82192,2023-11-01 07:43:31.452,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,-0.433884,0.279,6,Shopping,1,Clothing,1,Shopping,Other,0.0


In [140]:
train[train['y']==1]

,id2,id3,id4,id5,y,f1,f2,f5,f6,f7,...,f349_sin,time_since_last,session_event_count,offer_category,num_offer_categories,sub_category,num_sub_categories,previous_offer_category,previous_suboffer_category,no_of_clicks_previously
253,1002750,64187,2023-11-01 05:21:58.000709,2023-11-01,1,NaN,4.0,29.0,29.0,34.0,...,-0.433884,2.092709,49,Dining_Entertainment,2,Other,1,Shopping,Other,0.0
326,1002750,857184,2023-11-01 05:28:01.000106,2023-11-01,1,NaN,4.0,29.0,29.0,34.0,...,-0.433884,7.109106,122,Dining,1,Other,1,Shopping,Clothing,1.0
457,1003324,399752,2023-11-02 07:11:32.000766,2023-11-02,1,76.0,40.0,92.0,48.0,37.0,...,-0.974928,1.591766,44,Shopping,1,Apparel,1,Shopping,Other,0.0
527,1004168,498907,2023-11-01 14:01:41.000699,2023-11-01,1,NaN,NaN,38.0,NaN,NaN,...,-0.433884,12.449699,21,Travel,1,Hotel resorts,1,Shopping,Apparel_Clothing_Gifts_Jewelry_Other shopping,0.0
605,1004637,98107,2023-11-02 06:45:15.000821,2023-11-02,1,NaN,38.0,6.0,14.0,NaN,...,-0.974928,5.459821,21,Dining,1,Other,1,Shopping,Other,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124240,1910264,876665835,2023-11-01 16:08:14.000362,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,-0.433884,13.312362,14,Shopping,1,Other,1,Shopping,Clothing,0.0
124241,1910264,401464,2023-11-01 16:08:41.000508,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,-0.433884,27.000146,15,Shopping,1,Auto fuel,1,Shopping,Other,1.0
124317,1910264,81010,2023-11-01 16:19:50.000217,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,-0.433884,7.848217,91,Shopping,1,Other,1,Shopping,Other,2.0
124336,1910264,958700,2023-11-01 16:23:59.000189,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,-0.433884,10.611189,110,Entertainment,1,Other,1,Shopping,Jewelry,3.0


In [141]:
# Convert to integer
train['no_of_clicks_previously'] = train['no_of_clicks_previously'].astype(int)
val['no_of_clicks_previously'] = val['no_of_clicks_previously'].astype(int)

In [142]:
train

,id2,id3,id4,id5,y,f1,f2,f5,f6,f7,...,f349_sin,time_since_last,session_event_count,offer_category,num_offer_categories,sub_category,num_sub_categories,previous_offer_category,previous_suboffer_category,no_of_clicks_previously
0,1000624,5243695,2023-11-01 03:31:21.598,2023-11-01,0,56.0,NaN,33.0,16.0,33.0,...,-0.433884,NaN,1,Shopping,1,Other,1,missing,missing,0
1,1000624,97646,2023-11-01 03:31:21.618,2023-11-01,0,56.0,NaN,33.0,16.0,33.0,...,-0.433884,0.020,2,Shopping,1,Apparel,1,Shopping,Other,0
2,1000639,81010,2023-11-01 13:00:17.848,2023-11-01,0,44.0,NaN,24.0,46.0,42.0,...,-0.433884,NaN,1,Shopping,1,Other,1,missing,missing,0
3,1000639,830157,2023-11-01 13:00:20.013,2023-11-01,0,44.0,NaN,24.0,46.0,42.0,...,-0.433884,2.165,2,Shopping,1,Gifts,1,Shopping,Other,0
4,1000639,24840,2023-11-01 13:00:22.564,2023-11-01,0,44.0,NaN,24.0,46.0,42.0,...,-0.433884,2.551,3,Shopping,1,Clothing_Gifts_Jewelry,3,Shopping,Gifts,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124373,1910541,80107221,2023-11-01 05:46:56.017,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,-0.433884,1.993,3,Shopping,1,Other,1,Services,None,0
124374,1910541,224505,2023-11-01 07:43:02.618,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,-0.433884,6966.601,4,Shopping,1,Other,1,Shopping,Other,0
124375,1910541,5420674,2023-11-01 07:43:31.173,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,-0.433884,28.555,5,Shopping,1,Other,1,Shopping,Other,0
124376,1910541,82192,2023-11-01 07:43:31.452,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,-0.433884,0.279,6,Shopping,1,Clothing,1,Shopping,Other,0


## Feature 2: Pace x ctr
## Feature 3: session_event_count*f366
## Feature 4:session_count_x_ctr=session_evemt_count*f366

In [143]:
# Interaction between sequential and historical features
train['pace_x_ctr'] = train['time_since_last'] * train['f366'] 
train['session_count_x_ctr'] = train['session_event_count'] * train['f366']

In [144]:
train['pace_x_offer_popularity'] = train['time_since_last'] * train['f347']

In [145]:
train

,id2,id3,id4,id5,y,f1,f2,f5,f6,f7,...,offer_category,num_offer_categories,sub_category,num_sub_categories,previous_offer_category,previous_suboffer_category,no_of_clicks_previously,pace_x_ctr,session_count_x_ctr,pace_x_offer_popularity
0,1000624,5243695,2023-11-01 03:31:21.598,2023-11-01,0,56.0,NaN,33.0,16.0,33.0,...,Shopping,1,Other,1,missing,missing,0,NaN,0.000000,NaN
1,1000624,97646,2023-11-01 03:31:21.618,2023-11-01,0,56.0,NaN,33.0,16.0,33.0,...,Shopping,1,Apparel,1,Shopping,Other,0,0.000000,0.000000,0.001123
2,1000639,81010,2023-11-01 13:00:17.848,2023-11-01,0,44.0,NaN,24.0,46.0,42.0,...,Shopping,1,Other,1,missing,missing,0,NaN,0.005056,NaN
3,1000639,830157,2023-11-01 13:00:20.013,2023-11-01,0,44.0,NaN,24.0,46.0,42.0,...,Shopping,1,Gifts,1,Shopping,Other,0,0.010947,0.010112,0.130172
4,1000639,24840,2023-11-01 13:00:22.564,2023-11-01,0,44.0,NaN,24.0,46.0,42.0,...,Shopping,1,Clothing_Gifts_Jewelry,3,Shopping,Gifts,0,0.018441,0.021687,0.285766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124373,1910541,80107221,2023-11-01 05:46:56.017,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,Shopping,1,Other,1,Services,None,0,0.000000,0.000000,NaN
124374,1910541,224505,2023-11-01 07:43:02.618,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,Shopping,1,Other,1,Shopping,Other,0,0.000000,0.000000,480.712545
124375,1910541,5420674,2023-11-01 07:43:31.173,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,Shopping,1,Other,1,Shopping,Other,0,0.000000,0.000000,NaN
124376,1910541,82192,2023-11-01 07:43:31.452,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,Shopping,1,Clothing,1,Shopping,Other,0,0.000000,0.000000,0.021704


In [146]:
val['pace_x_offer_popularity'] = val['time_since_last'] * val['f347']
val['pace_x_ctr'] = val['time_since_last'] * val['f366'] 
val['session_count_x_ctr'] = val['session_event_count'] * val['f366']

# Feature 5

In [156]:
train = train.sort_values(['id2', 'id4'])

# 1. Create a column that only holds the timestamp when a click occurs
train['click_timestamp'] = train.loc[train['y'] == 1, 'id4']

# 2. Forward-fill this column to carry the last click time forward
# Now every row knows the time of the most recent click up to that point
train['last_click_timestamp'] = train.groupby('id2')['click_timestamp'].ffill()
train['last_click_timestamp'] = train.groupby('id2')['last_click_timestamp'].shift(1)

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_77493/3607711452.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train['last_click_timestamp'] = train.groupby('id2')['click_timestamp'].ffill()
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_77493/3607711452.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train['last_click_timestamp'] = train.groupby('id2')['last_click_timestamp'].shift(1)


In [157]:
train[train['y']==1]

,id2,id3,id4,id5,y,f1,f2,f5,f6,f7,...,sub_category,num_sub_categories,previous_offer_category,previous_suboffer_category,no_of_clicks_previously,pace_x_ctr,session_count_x_ctr,pace_x_offer_popularity,click_timestamp,last_click_timestamp
253,1002750,64187,2023-11-01 05:21:58.000709,2023-11-01,1,NaN,4.0,29.0,29.0,34.0,...,Other,1,Shopping,Other,0,0.008239,0.192913,0.138472,2023-11-01 05:21:58.000709,NaT
326,1002750,857184,2023-11-01 05:28:01.000106,2023-11-01,1,NaN,4.0,29.0,29.0,34.0,...,Other,1,Shopping,Clothing,1,0.053053,0.910448,0.526641,2023-11-01 05:28:01.000106,2023-11-01 05:21:58.000709
457,1003324,399752,2023-11-02 07:11:32.000766,2023-11-02,1,76.0,40.0,92.0,48.0,37.0,...,Apparel,1,Shopping,Other,0,0.028424,0.785714,0.115283,2023-11-02 07:11:32.000766,NaT
527,1004168,498907,2023-11-01 14:01:41.000699,2023-11-01,1,NaN,NaN,38.0,NaN,NaN,...,Hotel resorts,1,Shopping,Apparel_Clothing_Gifts_Jewelry_Other shopping,0,NaN,NaN,1.933568,2023-11-01 14:01:41.000699,NaT
605,1004637,98107,2023-11-02 06:45:15.000821,2023-11-02,1,NaN,38.0,6.0,14.0,NaN,...,Other,1,Shopping,Other,0,0.000000,0.000000,NaN,2023-11-02 06:45:15.000821,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124240,1910264,876665835,2023-11-01 16:08:14.000362,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,Other,1,Shopping,Clothing,0,0.055794,0.058676,NaN,2023-11-01 16:08:14.000362,NaT
124241,1910264,401464,2023-11-01 16:08:41.000508,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,Auto fuel,1,Shopping,Other,1,0.323743,0.179856,NaN,2023-11-01 16:08:41.000508,2023-11-01 16:08:14.000362
124317,1910264,81010,2023-11-01 16:19:50.000217,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,Other,1,Shopping,Other,2,0.094103,1.091127,0.500438,2023-11-01 16:19:50.000217,2023-11-01 16:08:41.000508
124336,1910264,958700,2023-11-01 16:23:59.000189,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,Other,1,Shopping,Jewelry,3,0.069218,0.717547,NaN,2023-11-01 16:23:59.000189,2023-11-01 16:19:50.000217


In [158]:
train['time_since_last_click'] = (train['id4'] - train['last_click_timestamp']).dt.total_seconds()
train.drop(columns=['click_timestamp', 'last_click_timestamp'], inplace=True)

In [159]:
train[train['y']==1]

,id2,id3,id4,id5,y,f1,f2,f5,f6,f7,...,num_offer_categories,sub_category,num_sub_categories,previous_offer_category,previous_suboffer_category,no_of_clicks_previously,pace_x_ctr,session_count_x_ctr,pace_x_offer_popularity,time_since_last_click
253,1002750,64187,2023-11-01 05:21:58.000709,2023-11-01,1,NaN,4.0,29.0,29.0,34.0,...,2,Other,1,Shopping,Other,0,0.008239,0.192913,0.138472,NaN
326,1002750,857184,2023-11-01 05:28:01.000106,2023-11-01,1,NaN,4.0,29.0,29.0,34.0,...,1,Other,1,Shopping,Clothing,1,0.053053,0.910448,0.526641,362.999397
457,1003324,399752,2023-11-02 07:11:32.000766,2023-11-02,1,76.0,40.0,92.0,48.0,37.0,...,1,Apparel,1,Shopping,Other,0,0.028424,0.785714,0.115283,NaN
527,1004168,498907,2023-11-01 14:01:41.000699,2023-11-01,1,NaN,NaN,38.0,NaN,NaN,...,1,Hotel resorts,1,Shopping,Apparel_Clothing_Gifts_Jewelry_Other shopping,0,NaN,NaN,1.933568,NaN
605,1004637,98107,2023-11-02 06:45:15.000821,2023-11-02,1,NaN,38.0,6.0,14.0,NaN,...,1,Other,1,Shopping,Other,0,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124240,1910264,876665835,2023-11-01 16:08:14.000362,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,1,Other,1,Shopping,Clothing,0,0.055794,0.058676,NaN,NaN
124241,1910264,401464,2023-11-01 16:08:41.000508,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,1,Auto fuel,1,Shopping,Other,1,0.323743,0.179856,NaN,27.000146
124317,1910264,81010,2023-11-01 16:19:50.000217,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,1,Other,1,Shopping,Other,2,0.094103,1.091127,0.500438,668.999709
124336,1910264,958700,2023-11-01 16:23:59.000189,2023-11-01,1,13.0,52.0,97.0,91.0,48.0,...,1,Other,1,Shopping,Jewelry,3,0.069218,0.717547,NaN,248.999972


In [160]:
val = val.sort_values(['id2', 'id4'])

# 1. Create a column that only holds the timestamp when a click occurs
val['click_timestamp'] = val.loc[val['y'] == 1, 'id4']

# 2. Forward-fill this column to carry the last click time forward
# Now every row knows the time of the most recent click up to that point
val['last_click_timestamp'] = val.groupby('id2')['click_timestamp'].ffill()
val['last_click_timestamp'] = val.groupby('id2')['last_click_timestamp'].shift(1)
val['time_since_last_click'] = (val['id4'] - val['last_click_timestamp']).dt.total_seconds()
val.drop(columns=['click_timestamp', 'last_click_timestamp'], inplace=True)

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_77493/94315800.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val['last_click_timestamp'] = val.groupby('id2')['click_timestamp'].ffill()
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_77493/94315800.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val['last_click_timestamp'] = val.groupby('id2')['last_click_timestamp'].shift(1)


In [161]:
val[val['y']==1]

,id2,id3,id4,id5,y,f1,f2,f5,f6,f7,...,num_offer_categories,sub_category,num_sub_categories,previous_suboffer_category,previous_offer_category,no_of_clicks_previously,pace_x_offer_popularity,pace_x_ctr,session_count_x_ctr,time_since_last_click
27,1000639,654480,2023-11-03 07:24:03.000710,2023-11-03,1,44.0,NaN,24.0,46.0,42.0,...,1,Gifts,1,Other shopping,Shopping,0,0.331322,0.026336,0.125392,NaN
117,1002750,70378,2023-11-03 14:59:51.000834,2023-11-03,1,NaN,4.0,29.0,29.0,34.0,...,1,Apparel,1,missing,missing,0,NaN,NaN,0.011152,NaN
197,1002750,69638,2023-11-03 15:01:15.000118,2023-11-03,1,NaN,4.0,29.0,29.0,34.0,...,1,Other,1,Electronics,Shopping,1,1.822701,0.075835,1.227273,83.999284
306,1004319,418984,2023-11-03 08:12:03.000240,2023-11-03,1,NaN,NaN,NaN,NaN,NaN,...,1,Other,1,Gifts_Other shopping,Shopping,0,NaN,0.044679,1.980952,NaN
376,1004319,159555,2023-11-03 08:13:03.000895,2023-11-03,1,NaN,NaN,NaN,NaN,NaN,...,1,Apparel_Clothing,2,Other,Shopping,1,NaN,0.984171,3.879032,60.000655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82850,1910212,35046,2023-11-03 08:07:38.000464,2023-11-03,1,13.0,51.0,34.0,91.0,97.0,...,1,Other,1,Other,Shopping,0,NaN,0.042742,0.378641,NaN
82878,1910212,9812,2023-11-03 08:08:52.000614,2023-11-03,1,13.0,51.0,34.0,91.0,97.0,...,1,Clothing_Jewelry,2,Other,Shopping,1,0.355502,0.071368,2.184783,74.000150
82900,1910212,25749,2023-11-03 08:10:32.000688,2023-11-03,1,13.0,51.0,34.0,91.0,97.0,...,1,Clothing,1,Communications_Computer services,Services,2,0.545500,0.233957,2.902174,100.000074
82906,1910261,721348,2023-11-03 12:11:52.000141,2023-11-03,1,13.0,34.0,38.0,98.0,45.0,...,1,Home garden,1,Other,Dining,0,12.009197,NaN,NaN,NaN


In [162]:
# save these into parquet files as small_train_feature_engineered_2
train.to_parquet('../Data/small_train_feature_engineered_2.parquet')
val.to_parquet('../Data/small_val_feature_engineered_2.parquet')